# Camera calibration

Notebook is based on tutorial by OpenCV project (https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html)

In [ ]:
!wget -O samples.csv https://raw.githubusercontent.com/ant-nik/semares/master/data/camera-calibration-std/position.csv

In [ ]:
import cv2
import logging
import io
import numpy


logger = logging.getLogger(__name__)
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)


def get_chess_corners_from_image(
        data: bytes, corners_x: int, corners_y: int) -> tuple[any, any]:
    # img = cv2.imread(fname)
    np_image = numpy.frombuffer(data, numpy.uint8)
    img = cv2.imdecode(np_image, cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (corners_x, corners_y), None)

    # If found, add object points, image points (after refining them)
    if ret != True:
        logger.error("Error, grid (%d, %d) is not found in image",
                     corners_x, corners_y)
        return None, gray

    corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)
    result_image = cv2.drawChessboardCorners(img, (7,7), corners, True)
    return corners2, result_image

In [ ]:
import plotly.express as plte
import requests
import logging
import pandas
import dataclasses


@dataclasses.dataclass
class Grid:
    name: str
    url: str
    corners_x: int
    corners_y: int
    corners: any
    image: any
    obj_points: any = None
    rvec: numpy.ndarray = None
    tvec: numpy.ndarray = None


logger = logging.getLogger(__name__)
base_url = "https://raw.githubusercontent.com/ant-nik/semares/master/data/camera-calibration-std/{}"

# set to None if all items are aceptable
acepted_items = [
    'image1_r.jpg',
    'image1_l.jpg',
    'image24_l.jpg',
    'image29_r.jpg',
    'image31_r.jpg',
    'image36_r.jpg',
    'image36_l.jpg'
]

sample_dataframe = pandas.read_csv('samples.csv', sep=',')
samples: list[Grid] = []
for index, item in sample_dataframe.iterrows():
    if acepted_items is not None and not item["image"] in acepted_items:
        continue
    url = base_url.format(item["image"])
    response = requests.get(url)
    if response.status_code != 200:
        logger.error("Can't read image %s from url %s", item["image"], url)
    # Draw and display the corners
    corners, image = get_chess_corners_from_image(
        data=response.content,
        corners_x=item["corners_x"],
        corners_y=item["corners_y"])
    if corners is None:
        logger.error("Can't find %d, %d conrenrs in %s",
                     item["corners_x"], item["corners_y"], item["image"])
    grid = Grid(
        name=item["image"],
        url=url,
        corners_x=item["corners_x"],
        corners_y=item["corners_y"],
        corners=corners,
        image=image
    )
    samples.append(grid)

In [ ]:
import logging


logger = logging.getLogger(__name__)
obj_points = []
img_points = []
objects = []
shapes = None
for sample in samples:
    if sample.corners is None:
        continue

    objp = numpy.zeros((sample.corners_x*sample.corners_y,3), numpy.float32)
    objp[:,:2] = numpy.mgrid[0:sample.corners_x,0:sample.corners_y].T.reshape(
        -1, 2)
    objp = objp
    img_shape = (sample.image.shape[1], sample.image.shape[0])
    if not shapes is None and shapes != img_shape:
        logger.error(
            "Image %s has specific shape %s that is not equals to previously shapes: %s",
            sample.name, str(img_shape), str(shapes)
            )
        continue
    if shapes is None:
        shapes = img_shape
    sample.obj_points = objp
    obj_points.append(objp)
    img_points.append(sample.corners)
    objects.append(sample)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(
    obj_points, img_points, shapes, None, None)
for i in range(0, len(objects)):
    objects[i].rvec = rvecs[i]
    objects[i].tvec = tvecs[i]
ret, mtx, dist

In [ ]:
w = shapes[0]
h = shapes[1]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))
(roi, newcameramtx)

In [ ]:
import logging


logger = logging.getLogger(__name__)
names = []
error_threshold = 0.15
axis = numpy.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)
color = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]

for sample in samples:
    if sample.corners is None:
        fig = plte.imshow(sample.image, title=sample.name)
        fig.show()
        logger.warning("Image %s has no detected chessboard corners, skipping...",
                       sample.name)
        continue
    dst = cv2.undistort(sample.image, mtx, dist, None, newcameramtx)
    # crop the image
    x, y, w, h = roi
    dst = dst[y:y + h, x:x + w]
    imgpoints, _ = cv2.projectPoints(
        sample.obj_points, sample.rvec, sample.tvec, mtx, dist)
    error = cv2.norm(sample.corners, imgpoints, cv2.NORM_L2)/len(imgpoints)
    if error < error_threshold:
        names.append(sample.name)
    axis2d, jac = cv2.projectPoints(axis, sample.rvec, sample.tvec, mtx, dist)
    origin = tuple([int(v) for v in sample.corners[0].ravel()])
    for i in range(0, 3):
        dst = cv2.line(dst, origin, tuple(
            [int(v) for v in axis2d[i].ravel()]), color[i], 5)
    fig = plte.imshow(
        dst, #sample.image,
        title=f"image {sample.name},crop[x,y,w,h]={roi},error={error}")
    fig.show()


In [ ]:
origin